In [1]:
#Ning Tientso
#25-Nov-2019
#SEC TP2 - RSA and SHA256

import numpy as np
import math
import random
import time
import struct

In [2]:
def Fast_Exponentiation(a, k, n):
    '''
    compute a**k mod n for big values k and n.
    '''
    
    #compute every exponent that is a power of 2
    x = a
    y = a if (k%2==1) else 1 #confirm that k is a power of 2, otherwise start at 1
    k_ = math.floor(k/2)
    
    while k_ > 0: #run while there is still powers to compute
        #x = x**2 % n
        x = pow(x,2,n)
        if (k_%2==1):
            y = x if y==1 else y*x%n #we either want y=a**n_i-1 or y=a**n_i
        k_ = math.floor(k_/2) #running count
    
    return y

In [3]:
def Faster_Exponentiation(a, k, n):
    '''
    The fast exponentiation implementation is above to show understanding,
    But due to uncertainty for which part is causing slowdown, used python's pow method instead,
    Please understand my situation
    '''
    return pow(a,k,n)

In [4]:
def Prime_Number_Generator():
    '''
    generate a big prime number n
    '''

    while(1): #keep running until we find what we want
        
        #generate a big random number n
        n = random.getrandbits(1000)
        
        #fermat prime test
        a = random.randint(2,n-2) #non inclusive n-1
        if Faster_Exponentiation(a, n-1, n) == 1: 
            return n #for big numbers, passing the test once is enough, don't you think?

In [5]:
a = Prime_Number_Generator()
print(a)

2449171020217963040628537465076145996263165790980931761853413230635853615472116818410050884081461397739915719572644414212324876381440032231600648440451184472537762808045519471757333421770983769137509149506105395057578348683962124879128398296293412169783207465213872230710754344225498925551850585885927


Yea... that's probably a prime...

In [6]:
def egcd(a, b):

    x = 0
    y = 1
    u = 1
    v = 0

    while a != 0:
        q = b//a
        r = b%a
        m = x-u*q
        n = y-v*q
        b = a
        a = r
        x = u
        y = v
        u = m
        v = n

    return b, x, y

In [7]:
#reminders on how to deal with strings, bit-blocks, and ints in python
rep_int = 2
rep_str = format(rep_int, "064b")
rep_bit = ord("a")
print(rep_int, rep_str, rep_bit)

#example convert message "hello" to bit blocks of size 16
def test_convert(message, block):
    bits = []
    for ch in message:
        bits.append(int(format(ord(ch), "0{0}b".format(block)),2))
    return bits

#example convert message from bits back to string
def test_revert(bits, block):
    message = ""
    for item in bits:
        message = message + chr(item)
    
    return message

a = test_convert("This is a test",32)
print(a)
print(test_revert(a, 32))

2 0000000000000000000000000000000000000000000000000000000000000010 97
[84, 104, 105, 115, 32, 105, 115, 32, 97, 32, 116, 101, 115, 116]
This is a test


In [8]:
from collections import deque

def rotate(seq, n):
    
    #rotate involves replacement of the value, so you join it to the very end
    seq.rotate(n)
    
    return "".join(seq)

def sum0(bits):
    
    #x2 according to assignment
    seq = deque(bits)
    x = int(rotate(seq.copy(), 2), 2) ^ int(rotate(seq.copy(), 13), 2) ^ int(rotate(seq.copy(), 22), 2)
    return x

def sum1(bits):
    
    #x1 according to assignmnet
    seq = deque(bits)
    x = int(rotate(seq.copy(), 6), 2) ^ int(rotate(seq.copy(), 11), 2)^int(rotate(seq.copy(),25),2)
    return x

def choose(x, y, z):
    
    #choosing x y or z
    return z^(x & (y^z))

def majority(x, y, z):
    
    #whichever one is the majority after x&z or x&y or y&z 
    return ((x|y)&z)|(x&y)

def padding(string):
    
    #padding by adding 1, and the requisite number of 0s to the string
    return string + "1" + "0"*(512-len(string)+1-len(format(len(string), '032b'))) + "".join(format(len(string),'032b'))

def sig0(bits):
    
    #s0 from the assignment
    seq = deque(bits)
    
    return int(rotate(seq.copy(), 7), 2)^int(rotate(seq.copy(),18),2)^int(bits,2) >> 3 #shift 3

def sig1(bits):
   

    #s1 from the assignment
    seq = deque(bits)
    
    return int(rotate(seq.copy(), 17), 2)^int(rotate(seq.copy(),19),2)^int(bits,2) >> 10 #shift 10

def SHA_256(string):
    
    #SHA256 proper?
    
    #first pad
    binary_string = padding("".join(format(ord(x), '08b') for x in string))
    split = []
    
    #break into many bits and pieces of 32 from 512
    for i in range(32, 513, 32):
        split.append(binary_string[i-32:i]) #block from beginning, and every block of size 32
    
    #IV
    h0 = 0x6a09e667 #a
    h1 = 0xbb67ae85 #b
    h2 = 0x3c6ef372 #c
    h3 = 0xa54ff53a #d
    h4 = 0x510e527f #e
    h5 = 0x9b05688c #f
    h6 = 0x1f83d9ab #g
    h7 = 0x5be0cd19 #h
    a,b,c,d,e,f,g,h = h0, h1, h2, h3, h4, h5, h6, h7
    
    #K constants
    k = (0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
       0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
       0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
       0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
       0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
       0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
       0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
       0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2 )
    
    #for 64 iterations as per the assignment
    for i in range(0, 64):
        
        if i > 15:
            split.append(format((sig1(split[i-2])+sig0(split[i-15])+int(split[i-7],2)+int(split[i-16],2))%2**32,'032b'))
        
        #The compression proper, temp1 and temp2 given from assignment, following use of MAJ and CH
        t1 = (h + sum1(format(e, '032b')) + choose(e,f,g) + k[i] + int(split[i],2))%2**32
        t2 = (sum0(format(a, '032b')) + majority(a,b,c))%2**32
        
        #set each a->h values
        h = g
        g = f
        f = e
        e = (d + t1) % 2 ** 32
        d = c
        c = b
        b = a
        a = (t1 + t2) % 2 ** 32
        
        #update each value mod 32
        h0 = (h0 + a) % 2 ** 32
        h1 = (h1 + b) % 2 ** 32
        h2 = (h2 + c) % 2 ** 32
        h3 = (h3 + d) % 2 ** 32
        h4 = (h4 + e) % 2 ** 32
        h5 = (h5 + f) % 2 ** 32
        h6 = (h6 + g) % 2 ** 32
        h7 = (h7 + h) % 2 ** 32
        
        return (hex(h0), hex(h1), hex(h2), hex(h3), hex(h4), hex(h5), hex(h6), hex(h7))


In [9]:
message = "Is this real life?"
print(message)
print(SHA_256(message))

Is this real life?
('0xaf858f28', '0x257194ec', '0xf7d6a1f7', '0xe1bee8ac', '0x33495595', '0xec13bb0b', '0xba894237', '0x7b64a6c4')


In [10]:
def gcd(a,b):
    #calculate the gcd between inputs a,b
    
    while b > 0: #note that if a<b then a%b == 0, while ends
        a,b = b, a%b
    
    return a

#def EEA already done

def inverse_mod(m, a):
    #calculates the inverse of a%m
    
    #get values from euclid's extended alg
    g, x, y = egcd(a, m)
    if g!=1:
        return None #inverse does not exist, account for this!!
    else:
        return x%m

def div_mod(m, a,b):
    #computes a/b mod m
    
    inverse = inverse_mod(m,b) #get inverse
    try:
        return (a*inverse)%m #division is just multiplying the inverse
    except:
        print("inverse did not exist")

#def Faster_Exponentiation(a,k,n) already done

def factors(x):
    #calculate the factors of x
    
    factors = []
    count = 2
    
    while x > 1:
        
        if x%count == 0: #everytime it is divisible by 2
            factors.append(count) #it is a factor
            x = x/count
        else:
            count+=1 #keep check
    
    return factors

def choose_e(totient):
    #calculate the 1 < e < totient, see if gcd(e, totient) == 1
    
    while(1):
        e = random.randint(2, totient)
        
        if gcd(e, totient) == 1:
            return e
        
#def Prime_Number_Generator() already done

def rsa_key():
    #p and q are BIG primes
    p = Prime_Number_Generator()
    q = Prime_Number_Generator()

    #n is the multiplication of these two BIG primes
    n = p*q

    #define phi,e, and d for decrypt
    phi=(p-1)*(q-1)
    e = choose_e(phi)
    d = inverse_mod(phi,e)
    
    return n,e,d

def rsa_enc(n,e,m):
    
    return Faster_Exponentiation(m, e, n)

def rsa_dec(n,d,c):
    
    return Faster_Exponentiation(c, d, n)

def RSA(message):
    #goes through RSA encrypt decrypt to test implementation
    
    #print original message value to test
    print("Original message: ",message)
    
    #convert message to numerical values
    BLOCK_SIZE = 32
    message_enc = test_convert(message, BLOCK_SIZE)
    print("Message encoded: ",message_enc)
    
    #gen keys
    n,e,d = rsa_key()
    
    #encryption for each chunk
    cipher_chunks=[]
    for block in message_enc:
        cipher = rsa_enc(n,e,block)
        cipher_chunks.append(cipher)
    
    print("Encrypted Text: ")
    print(cipher_chunks)

    #decryption
    plain_chunks = []
    for chunk in cipher_chunks:
        
        try:
            plain = rsa_dec(n,d,chunk) #for each chunk of the cipher, decrypt
        except:
            plain = -1
        
        plain_chunks.append(plain)
    
    #convert back to text
    plain_text = test_revert(plain_chunks, BLOCK_SIZE)
    print("Decrypted Text: ", plain_text)
    
    
    return plain_text == message #returns TRUE if encryption decryption worked

In [11]:
RSA("This is a test")

Original message:  This is a test
Message encoded:  [84, 104, 105, 115, 32, 105, 115, 32, 97, 32, 116, 101, 115, 116]
Encrypted Text: 
[23004365765317820541953179650809751236221220868561062345018825422873958628754076430969198985483737159141968394936910631111138699926869216601616715095589782083109208044078807812978357378555701083914105766068697460015465259951991907122525893693462572530119253844896021710136809916863218757566893621660834339134405080555639345607836084771252948269077367047352827269202716222512047765061344607599077479608172924665787629095249448253367877058864721385313023220187949032337984491034907074816946990350900996333712491313498205075294617252015165200136912301809794242959007525288056889581829402194724137529338504, 71933412471820120410235716094762366392036076177896521700375016244271801246507192722379068832106268937246189865900222955989128903067325437571328416719623228390187900245319659278193322630818265570628425477548909110592635085945497349576374668246135584185383683049

Decrypted Text:  This is a test


True